# Props Data Organizer

## Definitions of Formulas Used in This Notebook

### Estimated Value (EV)
EV is the average amount you can expect to win or lose per bet if you placed the same bet many times. It helps identify profitable betting opportunities by comparing the expected return to the risk involved.

**Formula:**
$$
\text{EV} = (\text{Probability of Winning} \times \text{Profit if Win}) - (\text{Probability of Losing} \times \text{Loss if Lose})
$$

### Kelly Criterion
The Kelly Criterion is a formula used to determine the optimal size of a series of bets. It aims to maximize the logarithm of wealth, balancing the trade-off between risk and reward. The formula considers both the probability of winning and the odds offered, guiding you on how much of your bankroll to wager on each bet.

**Formula:**
$$
\text{Kelly Fraction} = \frac{(\text{Probability of Winning} \times (\text{Odds} + 1)) - 1}{\text{Odds}}
$$

In [2]:
import pandas as pd 
import numpy as np
import time
import requests
from NBAData.gambling import *

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [ ]:
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()
prizePicks = nba_props.dataframe
prizePicks.to_csv('PROPS_DATA/Playoffs_US(6_15_25).csv')
prizePicks.head(10)

Scraping Odds API...
Organizing Data...


,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,DraftKings,player_points,Jalen Williams,Over,22.5,-110
3,DraftKings,player_points,Jalen Williams,Under,22.5,-120
4,DraftKings,player_points,Pascal Siakam,Over,19.5,-105
5,DraftKings,player_points,Pascal Siakam,Under,19.5,-125
6,DraftKings,player_points,Tyrese Haliburton,Over,16.5,-125
7,DraftKings,player_points,Tyrese Haliburton,Under,16.5,-105
8,DraftKings,player_points,Chet Holmgren,Over,15.5,-105
9,DraftKings,player_points,Chet Holmgren,Under,15.5,-125


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [3]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
all_bookmakers = pd.read_csv('./PROPS_DATA/Playoffs_US(6_15_25).csv')
prop_type = all_bookmakers['CATEGORY'].unique()
print(prop_type)
all_bookmakers

['player_points' 'player_rebounds' 'player_assists' 'player_threes'
 'player_blocks' 'player_steals' 'player_points_rebounds_assists'
 'player_points_rebounds' 'player_points_assists'
 'player_rebounds_assists' 'player_turnovers' 'player_blocks_steals']


,Unnamed: 0,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,2,DraftKings,player_points,Jalen Williams,Over,22.5,-110
3,3,DraftKings,player_points,Jalen Williams,Under,22.5,-120
4,4,DraftKings,player_points,Pascal Siakam,Over,19.5,-105
...,...,...,...,...,...,...,...
1827,1827,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-175
1828,1828,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,150
1829,1829,DraftKings,player_blocks_steals,Isaiah Hartenstein,Under,1.5,-195
1830,1830,DraftKings,player_blocks_steals,Obi Toppin,Over,1.5,154


In [5]:
import scipy.stats as stats


def single_bet(data, bookmakers, category='player_points', stat_line='PTS'):  
    Props = bookmakers[['NAME', 'BOOKMAKER', 'CATEGORY', 'LINE', 'OVER/UNDER', 'ODDS']].loc[bookmakers['CATEGORY'] == category]
    results = []

    for idx, row in Props.iterrows():
        name = row['NAME']
        bookmaker = row['BOOKMAKER']
        line = row['LINE']
        over_under = row['OVER/UNDER']
        odds = row['ODDS']

        # Get combined or single stat values
        stat_values = get_combined_stat_values(data, name, stat_line)
        if stat_values.empty:
            continue

        mean = round(stat_values.mean(), 2)
        std = round(stat_values.std(), 2)
        if std == 0:
            continue

        # Z-score-based probability
        z_score = (line - mean) / std
        norm_prob = 1 - stats.norm.cdf(z_score)
        if over_under == "Under":
            norm_prob = 1 - norm_prob

        # Monte Carlo simulation
        sim_points = np.random.normal(mean, std, 10000)
        sim_prob = np.mean(sim_points > line)
        if over_under == "Under":
            sim_prob = 1 - sim_prob

        # Weighted average of sim and norm prob
        final_prob = 0.8 * sim_prob + 0.2 * norm_prob
        
        # EV calculation
        stake = 100
        profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
        payout = stake + profit
        ev = (final_prob * profit) - ((1 - final_prob) * stake)

        kelly = kelly_criterion(final_prob, payout, stake)

        # Calculate fair odds
        try:
            fair_odds = fairProb(bookmakers, name, line, category, over_under)
        except ValueError as e:
            fair_odds = None

        results.append({
            'NAME': name,
            'BOOKMAKER': bookmaker,
            'CATEGORY': category,
            'LINE': line,
            'OVER/UNDER': over_under,
            'ODDS': odds,
            'FAIR ODDS': fair_odds,
            'SIM PROB': round(final_prob, 3),
            'EV': round(ev, 2),
            'KELLY CRITERION': kelly,
        })

    return pd.DataFrame(results)

In [6]:
# Dictionary mapping prop categories to their stat columns
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_turnovers': 'TOV',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_blocks_steals': 'BLK+STL'
}

all_results = []

for category, stat in propDict.items():
    print(f"Processing {category}...")
    results = single_bet(data, all_bookmakers, category=category, stat_line=stat)
    all_results.append(results)

combined_results = pd.concat(all_results, ignore_index=True)

final_results = combined_results.sort_values(by='EV', ascending=False).reset_index(drop=True)

print("\nTop 15 highest EV bets across all prop types:")

# Save to CSV
final_results.to_csv('PROPS_EV/SingleBets(6_15_25).csv')
final_results.head(15)

Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_turnovers...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_blocks_steals...

Top 15 highest EV bets across all prop types:


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,FAIR ODDS,SIM PROB,EV,KELLY CRITERION
0,Obi Toppin,Bovada,player_rebounds,4.5,Under,165,190,0.650,72.38,0.6484
1,Pascal Siakam,DraftKings,player_turnovers,1.5,Under,150,174,0.685,71.33,0.6832
2,Aaron Nesmith,Bovada,player_threes,2.5,Over,190,228,0.548,58.97,0.5458
3,T.J. McConnell,BetMGM,player_threes,0.5,Over,290,338,0.404,57.70,0.4023
4,Obi Toppin,Bovada,player_rebounds_assists,6.5,Under,110,122,0.744,56.32,0.7421
5,T.J. McConnell,BetMGM,player_threes,0.5,Over,290,338,0.398,55.30,0.3961
6,Aaron Nesmith,Bovada,player_threes,2.5,Over,190,228,0.535,55.11,0.5324
7,Isaiah Hartenstein,BetRivers,player_points,6.5,Over,117,129,0.705,52.90,0.7021
8,Isaiah Hartenstein,DraftKings,player_points,6.5,Over,114,129,0.710,52.01,0.7078
9,Obi Toppin,DraftKings,player_rebounds_assists,6.5,Under,105,122,0.741,51.93,0.7386


## PrizePicks & Underdog
## 2 leg EVs since we cant do single bets on those platforms

### Functions used to grab the ev and kelly

In [5]:
from scipy.stats import norm

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))

def get_combined_stat_values(data, player_name, stat_line):
    """Helper function to get stat values for single or combined stats"""
    df = data.loc[data['PLAYER_NAME'] == player_name]
    
    if df.empty:
        return pd.Series()
    
    if '+' in stat_line:
        stat_parts = stat_line.split('+')
        return df[stat_parts].sum(axis=1)
    else:
        return df[stat_line]

def prizePicksPairsEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=300, weight=0.8):
    def monte_carlo_sim(name1, name2, data, prizePicks1, prizePicks2, simulations, stake, payout, weight, stat_line1, stat_line2):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]

        if df1.empty or df2.empty:
            return None

        line1 = prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0]

        stat1_values = get_combined_stat_values(data, name1, stat_line1)
        stat2_values = get_combined_stat_values(data, name2, stat_line2)
        
        if len(stat1_values) == 0 or len(stat2_values) == 0:
            return None

        # Calculate mean and std 
        mean1, std1 = stat1_values.mean(), stat1_values.std()
        mean2, std2 = stat2_values.mean(), stat2_values.std()
        
        if std1 == 0 or std2 == 0:
            return None

        # Monte Carlo simulation using normal distribution
        sim1 = np.random.normal(mean1, std1, simulations)
        sim2 = np.random.normal(mean2, std2, simulations)

        over1, under1 = sim1 > line1, sim1 <= line1
        over2, under2 = sim2 > line2, sim2 <= line2

        mc_probs = {
            'OVER/OVER': np.mean(over1 & over2),
            'UNDER/UNDER': np.mean(under1 & under2),
            'OVER/UNDER': np.mean(over1 & under2),
            'UNDER/OVER': np.mean(under1 & over2),
        }

        z_over1 = zscore_prob(mean1, std1, line1, side='over')
        z_under1 = 1 - z_over1
        z_over2 = zscore_prob(mean2, std2, line2, side='over')
        z_under2 = 1 - z_over2

        z_probs = {
            'OVER/OVER': z_over1 * z_over2,
            'UNDER/UNDER': z_under1 * z_under2,
            'OVER/UNDER': z_over1 * z_under2,
            'UNDER/OVER': z_under1 * z_over2,
        }

        final_probs = {
            k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
            for k in mc_probs
        }

        evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}

        return {
            'OVER/OVER': {'prob': round(final_probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
            'UNDER/UNDER': {'prob': round(final_probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
            'OVER/UNDER': {'prob': round(final_probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
            'UNDER/OVER': {'prob': round(final_probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
        }

    all_pairs = []
    seen = set()

    # Iterate through all categories for player 1
    for cat1 in propDict:
        print(f"Processing {cat1}...")
        prizePicks1 = prizePicks[prizePicks['CATEGORY'] == cat1]
        stat_line1 = propDict[cat1]
        
        # Compare against all categories (including same category)
        for cat2 in propDict:
            print(f"Processing {cat2}...")
            prizePicks2 = prizePicks[prizePicks['CATEGORY'] == cat2]
            stat_line2 = propDict[cat2]
            
            for name1 in prizePicks1['NAME'].unique():
                for name2 in prizePicks2['NAME'].unique():
                    if name1 == name2:
                        continue
                        
                    # Creates unique keys
                    combo_key = tuple(sorted([
                        (name1, cat1),
                        (name2, cat2)
                    ]))
                    
                    if combo_key in seen:
                        continue
                    seen.add(combo_key)

                    sim_result = monte_carlo_sim(
                        name1, name2, data, 
                        prizePicks1, prizePicks2,
                        simulations, stake, payout, weight,
                        stat_line1, stat_line2
                    )
                    
                    if sim_result is None:
                        continue

                    evs = {
                        'OVER/OVER': sim_result['OVER/OVER']['ev'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['ev'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['ev']
                    }
                    
                    probs = {
                        'OVER/OVER': sim_result['OVER/OVER']['prob'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['prob'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['prob']
                    }

                    best_type = max(evs, key=evs.get)
                    best_ev = evs[best_type]
                    best_prob = probs[best_type]

                    all_pairs.append({
                        'PLAYER 1': name1,
                        'CATEGORY 1': stat_line1,
                        'PLAYER 1 LINE': prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0],
                        'PLAYER 2': name2,
                        'CATEGORY 2': stat_line2,
                        'PLAYER 2 LINE': prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0],
                        'TYPE': best_type,
                        'EV': round(best_ev, 2),
                        'PROBABILITY': round(best_prob, 4),
                        'KELLY CRITERION': kelly_criterion(best_prob, payout, stake)
                    })

    return pd.DataFrame(all_pairs)

In [ ]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
pp_data = pd.read_csv('PROPS_DATA/Playoffs_DFS(6_15_25).csv')
prizePicks = pp_data[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[pp_data['BOOKMAKER'] == 'PrizePicks']
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}

# Run the analysis
results = prizePicksPairsEV(data, pp_data, propDict)
results.to_csv('PROPS_EV/PrizePicksPairs(6_15_25).csv')

results.sort_values(by='KELLY CRITERION', ascending=False).head(15)


Processing player_points...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers...
Processing player_rebounds...
Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_turnovers...
Processing player_assist

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
2035,Isaiah Hartenstein,PTS,5.5,Obi Toppin,REB+AST,7.0,OVER/UNDER,91.07,0.6369,0.6351
277,Isaiah Hartenstein,PTS,5.5,Obi Toppin,REB,5.5,OVER/UNDER,88.74,0.6291,0.6272
6970,Shai Gilgeous-Alexander,FGM,12.5,Obi Toppin,REB+AST,7.0,UNDER/UNDER,86.82,0.6227,0.6208
6996,Isaiah Hartenstein,FGM,2.5,Obi Toppin,REB+AST,7.0,OVER/UNDER,85.37,0.6179,0.6160
903,Isaiah Hartenstein,PTS,5.5,Shai Gilgeous-Alexander,FGM,12.5,OVER/UNDER,82.47,0.6082,0.6062
3734,Alex Caruso,REB,4.0,Obi Toppin,REB+AST,7.0,UNDER/UNDER,82.25,0.6075,0.6055
2643,Obi Toppin,REB,5.5,Shai Gilgeous-Alexander,FGM,12.5,UNDER/UNDER,81.75,0.6058,0.6038
2645,Obi Toppin,REB,5.5,Isaiah Hartenstein,FGM,2.5,UNDER/OVER,81.69,0.6056,0.6036
2174,Obi Toppin,REB,5.5,Alex Caruso,REB,4.0,UNDER/UNDER,79.28,0.5976,0.5956
286,Isaiah Hartenstein,PTS,5.5,Alex Caruso,REB,4.0,OVER/UNDER,77.95,0.5932,0.5912


## 3 leg EVs for prizepicks and underdogs

In [33]:

def prizePicksTriosEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=600, weight=0.8):
    """Optimized version with pre-computed lookups and parallel processing"""
    
    # 1. Pre-compute player statistics for all stat lines
    print("Pre-computing player statistics...")
    player_stats_cache = {}
    
    for stat_line in set(propDict.values()):
        for player in data['PLAYER_NAME'].unique():
            stat_values = get_combined_stat_values(data, player, stat_line)
            if not stat_values.empty:
                mean, std = stat_values.mean(), stat_values.std()
                if std > 0:  # Only store valid stats
                    player_stats_cache[(player, stat_line)] = {
                        'mean': mean, 'std': std, 'values': stat_values
                    }
    
    # 2. Pre-compute prize picks lookups by category and player
    print("Pre-computing prize picks lookups...")
    prizepicks_lookup = {}
    for category in propDict.keys():
        category_data = prizePicks[prizePicks['CATEGORY'] == category]
        for _, row in category_data.iterrows():
            prizepicks_lookup[(row['NAME'], category)] = row['LINE']
    
    # 3. Pre-generate valid combinations (avoid duplicates upfront)
    print("Generating valid combinations...")
    valid_combinations = []
    
    available_players = set()
    for category in propDict.keys():
        category_players = prizePicks[prizePicks['CATEGORY'] == category]['NAME'].unique()
        for player in category_players:
            stat_line = propDict[category]
            if (player, stat_line) in player_stats_cache:
                available_players.add((player, category))
    
    available_players = list(available_players)
    
    # Generate all valid trios
    for i in range(len(available_players)):
        for j in range(i + 1, len(available_players)):
            for k in range(j + 1, len(available_players)):
                player1, cat1 = available_players[i]
                player2, cat2 = available_players[j]
                player3, cat3 = available_players[k]
                
                # Skip if any players are the same
                if player1 == player2 or player1 == player3 or player2 == player3:
                    continue
                
                # Check if all required data exists
                stat_line1, stat_line2, stat_line3 = propDict[cat1], propDict[cat2], propDict[cat3]
                
                if (player1, stat_line1) in player_stats_cache and \
                   (player2, stat_line2) in player_stats_cache and \
                   (player3, stat_line3) in player_stats_cache and \
                   (player1, cat1) in prizepicks_lookup and \
                   (player2, cat2) in prizepicks_lookup and \
                   (player3, cat3) in prizepicks_lookup:
                    
                    valid_combinations.append({
                        'players': [player1, player2, player3],
                        'categories': [cat1, cat2, cat3],
                        'stat_lines': [stat_line1, stat_line2, stat_line3],
                        'lines': [
                            prizepicks_lookup[(player1, cat1)],
                            prizepicks_lookup[(player2, cat2)],
                            prizepicks_lookup[(player3, cat3)]
                        ]
                    })
    
    # 4. Process combinations with Monte Carlo simulations - THREADED VERSION
    print(f"Running Monte Carlo simulations for {len(valid_combinations)} combinations...")
    
    def process_combination(combo):
        """Process a single combination"""
        players, categories, stat_lines, lines = combo['players'], combo['categories'], combo['stat_lines'], combo['lines']
        
        try:
            # Get cached statistics
            stats = [player_stats_cache[(players[j], stat_lines[j])] for j in range(3)]
            means = [s['mean'] for s in stats]
            stds = [s['std'] for s in stats]
            
            # Generate simulations for each player individually
            sim1 = np.random.normal(means[0], stds[0], simulations)
            sim2 = np.random.normal(means[1], stds[1], simulations)
            sim3 = np.random.normal(means[2], stds[2], simulations)
            
            # Calculate over/under results
            over1, under1 = sim1 > lines[0], sim1 <= lines[0]
            over2, under2 = sim2 > lines[1], sim2 <= lines[1]
            over3, under3 = sim3 > lines[2], sim3 <= lines[2]
            
            # Calculate Monte Carlo probabilities
            mc_probs = {
                'OVER/OVER/OVER': np.mean(over1 & over2 & over3),
                'OVER/OVER/UNDER': np.mean(over1 & over2 & under3),
                'OVER/UNDER/OVER': np.mean(over1 & under2 & over3),
                'OVER/UNDER/UNDER': np.mean(over1 & under2 & under3),
                'UNDER/OVER/OVER': np.mean(under1 & over2 & over3),
                'UNDER/OVER/UNDER': np.mean(under1 & over2 & under3),
                'UNDER/UNDER/OVER': np.mean(under1 & under2 & over3),
                'UNDER/UNDER/UNDER': np.mean(under1 & under2 & under3),
            }
            
            # Calculate z-score probabilities
            z_over1 = zscore_prob(means[0], stds[0], lines[0], side='over')
            z_under1 = 1 - z_over1
            z_over2 = zscore_prob(means[1], stds[1], lines[1], side='over')
            z_under2 = 1 - z_over2
            z_over3 = zscore_prob(means[2], stds[2], lines[2], side='over')
            z_under3 = 1 - z_over3
            
            z_probs = {
                'OVER/OVER/OVER': z_over1 * z_over2 * z_over3,
                'OVER/OVER/UNDER': z_over1 * z_over2 * z_under3,
                'OVER/UNDER/OVER': z_over1 * z_under2 * z_over3,
                'OVER/UNDER/UNDER': z_over1 * z_under2 * z_under3,
                'UNDER/OVER/OVER': z_under1 * z_over2 * z_over3,
                'UNDER/OVER/UNDER': z_under1 * z_over2 * z_under3,
                'UNDER/UNDER/OVER': z_under1 * z_under2 * z_over3,
                'UNDER/UNDER/UNDER': z_under1 * z_under2 * z_under3,
            }
            
            # Combine probabilities
            final_probs = {
                k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
                for k in mc_probs
            }
            
            # Calculate EVs
            evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}
            
            # Find best combination
            best_type = max(evs, key=evs.get)
            best_ev = evs[best_type]
            best_prob = final_probs[best_type]
            
            return {
                'players': players,
                'categories': categories,
                'stat_lines': stat_lines,
                'lines': lines,
                'best_type': best_type,
                'best_ev': best_ev,
                'best_prob': best_prob
            }
            
        except Exception as e:
            print(f"Error processing combination {players}: {e}")
            return None
    
    # Process combinations in parallel using threads
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import multiprocessing as mp
    
    results = []
    max_workers = min(mp.cpu_count(), len(valid_combinations))
    
    print(f"Processing {len(valid_combinations)} combinations with {max_workers} threads...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_combo = {
            executor.submit(process_combination, combo): i 
            for i, combo in enumerate(valid_combinations)
        }
        
        # Collect results as they complete
        completed = 0
        for future in as_completed(future_to_combo):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
                completed += 1
                
                # Show progress every 100 completions
                if completed % 100 == 0:
                    print(f"Completed {completed}/{len(valid_combinations)} combinations")
                    
            except Exception as e:
                print(f"Error in future: {e}")
    
    print(f"Successfully processed {len(results)} combinations")
    
    # 5. Build final results
    print("Building final results...")
    all_trios = []
    
    for result in results:
        all_trios.append({
            'PLAYER 1': result['players'][0],
            'CATEGORY 1': result['stat_lines'][0], 
            'PLAYER 1 LINE': result['lines'][0],
            'PLAYER 2': result['players'][1],
            'CATEGORY 2': result['stat_lines'][1],
            'PLAYER 2 LINE': result['lines'][1],
            'PLAYER 3': result['players'][2],
            'CATEGORY 3': result['stat_lines'][2],
            'PLAYER 3 LINE': result['lines'][2],
            'TYPE': result['best_type'],
            'EV': round(result['best_ev'], 2),
            'PROBABILITY': round(result['best_prob'], 4),
            'KELLY CRITERION': kelly_criterion(result['best_prob'], payout, stake)
        })
    
    return pd.DataFrame(all_trios)

In [34]:

data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv('PROPS_DATA/Playoffs_DFS(6_15_25).csv')
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}

trio_results = prizePicksTriosEV(data, prizePicks, propDict, payout=600).sort_values('KELLY CRITERION', ascending=False).reset_index(drop=True)
trio_results

# trio_results.to_csv('PROPS_EV/PrizePicksTrios(6_15_25).csv')

Pre-computing player statistics...
Pre-computing prize picks lookups...
Generating valid combinations...
Running Monte Carlo simulations for 399804 combinations...
Processing 399804 combinations with 8 threads...
Completed 100/399804 combinations
Completed 200/399804 combinations
Completed 300/399804 combinations
Completed 400/399804 combinations
Completed 500/399804 combinations
Completed 600/399804 combinations
Completed 700/399804 combinations
Completed 800/399804 combinations
Completed 900/399804 combinations
Completed 1000/399804 combinations
Completed 1100/399804 combinations
Completed 1200/399804 combinations
Completed 1300/399804 combinations
Completed 1400/399804 combinations
Completed 1500/399804 combinations
Completed 1600/399804 combinations
Completed 1700/399804 combinations
Completed 1800/399804 combinations
Completed 1900/399804 combinations
Completed 2000/399804 combinations
Completed 2100/399804 combinations
Completed 2200/399804 combinations
Completed 2300/399804 comb

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,PLAYER 3,CATEGORY 3,PLAYER 3 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Isaiah Hartenstein,FGM,2.5,Obi Toppin,REB,5.5,Alex Caruso,REB,4.0,OVER/UNDER/UNDER,178.72,0.4645,0.4635
1,Isaiah Hartenstein,FGM,2.5,Obi Toppin,REB,5.5,Shai Gilgeous-Alexander,FGM,12.5,OVER/UNDER/UNDER,177.94,0.4632,0.4622
2,Obi Toppin,REB,5.5,Alex Caruso,REB,4.0,Shai Gilgeous-Alexander,FGM,12.5,UNDER/UNDER/UNDER,175.88,0.4598,0.4587
3,Isaiah Hartenstein,FGM,2.5,Obi Toppin,REB+AST,6.5,Shai Gilgeous-Alexander,FGM,12.5,OVER/UNDER/UNDER,166.34,0.4439,0.4428
4,Isaiah Hartenstein,FGM,2.5,Alex Caruso,REB,4.0,Shai Gilgeous-Alexander,FGM,12.5,OVER/UNDER/UNDER,165.91,0.4432,0.4421
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399799,Chet Holmgren,PTS+REB+AST,25.5,Alex Caruso,AST,2.5,Myles Turner,REB,5.0,UNDER/UNDER/OVER,-22.83,0.1286,0.1269
399800,Chet Holmgren,PTS+REB,24.5,Luguentz Dort,REB+AST,4.5,Bennedict Mathurin,REB,2.5,UNDER/UNDER/UNDER,-22.87,0.1286,0.1268
399801,Chet Holmgren,PTS+REB+AST,25.5,Myles Turner,FG3M,1.5,Luguentz Dort,REB+AST,4.5,UNDER/OVER/UNDER,-23.02,0.1283,0.1265
399802,Chet Holmgren,PTS+REB+AST,25.5,Cason Wallace,PTS,5.5,Luguentz Dort,REB,3.5,OVER/UNDER/OVER,-23.59,0.1273,0.1256


⚙️ Hybrid Approach Recommended
For NBA prop betting, a hybrid workflow often works best:
- Train an ML model to predict the expected prop value using features like usage rate, matchup context, rest, etc.
- Simulate multiple game scenarios via Monte Carlo, using your ML model’s predictive distribution (e.g., add residual noise or use a Bayesian framework).
- Bootstrap on your historical residuals to calibrate confidence intervals around those predictions.

This combines:
ML’s ability to capture complex relationships,
Monte Carlo’s capacity for modeling variability and outputting a full prob. distribution,
Bootstrap’s non-parametric estimate of uncertainty—all necessary to assess value and risk correctly 

🗓️ Practical Takeaway
Need a full probability distribution for value + risk? MC sim driven by your ML model is essential.


📌 Final Recommendation
Use Machine Learning to predict the mean prop, then Monte Carlo to simulate possible outcomes (using those ML predictions with added variance), and Bootstrap to quantify your confidence in those outcomes. This hybrid setup offers the most accurate, actionable probabilities for NBA prop betting.
